# TMDB Enrichment: budget / revenue / adult
This notebook enriches the MovieLens training datasets with a small set of **external movie-level features** from the Kaggle TMDB dataset:
- `budget`
- `revenue`
- `adult`

We avoid calling external APIs by joining via **IMDb id** (MovieLens `links.csv` → TMDB `imdb_id`).

[Kaggle link](https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/data)

Outputs:
- `train_prepared_v3_tmdb.parquet`
- `val_prepared_v3_tmdb.parquet`
- `test_prepared_v3_tmdb.parquet`


## 0) Imports

In [1]:
import pandas as pd
import numpy as np


## 1) Load MovieLens prepared splits

In [10]:
train = pd.read_parquet("../data/processed/train_prepared.parquet")
val = pd.read_parquet("../data/processed/val_prepared.parquet")
test = pd.read_parquet("../data/processed/test_prepared.parquet")

print("rows:", {"train": len(train), "val": len(val), "test": len(test)})
train[["userId","movieId","rating","high_rating"]].head()

rows: {'train': 4490259, 'val': 599327, 'test': 846774}


,userId,movieId,rating,high_rating
0,348,6163,4.0,1
1,348,45722,4.0,1
2,348,53125,4.0,1
3,348,54259,4.0,1
4,359,316,4.0,1


## 2) Load MovieLens links.csv and build IMDb ids
MovieLens `links.csv` maps `movieId` → `imdbId` (numeric). TMDB uses `imdb_id` strings like `tt0114709`.

We create:
- `imdb_id` = `'tt' + imdbId padded to 7 digits`


In [2]:
links = pd.read_csv("../data/raw/movielens-20m-dataset/link.csv")  
links["imdbId"] = pd.to_numeric(links["imdbId"], errors="coerce").astype("Int64")
links["imdb_id"] = links["imdbId"].apply(lambda x: f"tt{int(x):07d}" if pd.notna(x) else pd.NA)

print("links:", links.shape)
links[["movieId","imdbId","imdb_id"]].head(10)


links: (27278, 4)


,movieId,imdbId,imdb_id
0,1,114709,tt0114709
1,2,113497,tt0113497
2,3,113228,tt0113228
3,4,114885,tt0114885
4,5,113041,tt0113041
5,6,113277,tt0113277
6,7,114319,tt0114319
7,8,112302,tt0112302
8,9,114576,tt0114576
9,10,113189,tt0113189


## 3) Load TMDB dataset and keep only needed columns
Download the Kaggle dataset locally and point `TMDB_PATH` to the CSV file.

We keep only:
- `imdb_id`
- `budget`
- `revenue`
- `adult`


In [3]:
TMDB_PATH = "../data/raw/imd_extra_info/TMDB_movie_dataset_v11.csv"
tmdb = pd.read_csv(
    TMDB_PATH,
    usecols=["imdb_id", "budget", "revenue", "adult"],
    low_memory=False
)

print("tmdb:", tmdb.shape)
tmdb.head()


tmdb: (1361438, 4)


,revenue,adult,budget,imdb_id
0,825532764,False,160000000,tt1375666
1,701729206,False,165000000,tt0816692
2,1004558444,False,185000000,tt0468569
3,2923706026,False,237000000,tt0499549
4,1518815515,False,220000000,tt0848228


In [7]:
tmdb[tmdb.imdb_id=='tt0000000']

,revenue,adult,budget,imdb_id
636163,NaN,0.0,NaN,tt0000000


## 4) Clean TMDB features
Typical issues:
- `budget` / `revenue` may be 0 for unknown (treat as missing)
- duplicates by `imdb_id` (we keep max budget/revenue; adult -> max)

We also add log-transformed versions to stabilize heavy tails:
- `log1p_budget`
- `log1p_revenue`


In [4]:
tmdb["budget"] = pd.to_numeric(tmdb["budget"], errors="coerce")
tmdb["revenue"] = pd.to_numeric(tmdb["revenue"], errors="coerce")

tmdb.loc[tmdb["budget"] <= 0, "budget"] = np.nan
tmdb.loc[tmdb["revenue"] <= 0, "revenue"] = np.nan

def to_adult_flag(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, bool):
        return int(x)
    s = str(x).strip().lower()
    if s in ("true", "t", "1", "yes", "y"):
        return 1
    if s in ("false", "f", "0", "no", "n"):
        return 0
    return np.nan

tmdb["adult"] = tmdb["adult"].apply(to_adult_flag).astype("float32")

tmdb_agg = (
    tmdb.groupby("imdb_id", as_index=False)
    .agg(
        budget=("budget", "max"),
        revenue=("revenue", "max"),
        adult=("adult", "max"),
    )
)

tmdb_agg["log1p_budget"] = np.log1p(tmdb_agg["budget"])
tmdb_agg["log1p_revenue"] = np.log1p(tmdb_agg["revenue"])

print("tmdb_agg:", tmdb_agg.shape)
tmdb_agg.head()


tmdb_agg: (651729, 6)


,imdb_id,budget,revenue,adult,log1p_budget,log1p_revenue
0,tt0000000,NaN,NaN,0.0,NaN,NaN
1,tt0000001,NaN,NaN,0.0,NaN,NaN
2,tt0000002,NaN,NaN,0.0,NaN,NaN
3,tt0000003,NaN,NaN,0.0,NaN,NaN
4,tt0000004,NaN,NaN,0.0,NaN,NaN


In [6]:
tmdb_agg[tmdb_agg.imdb_id=='tt1375666']

,imdb_id,budget,revenue,adult,log1p_budget,log1p_revenue
330088,tt1375666,160000000.0,825532764.0,0.0,18.890684,20.53154


## 5) Join TMDB → MovieLens movieId via IMDb
We map:
`movieId` → `imdb_id` (MovieLens links) → TMDB features.


In [8]:
movie_tmdb = (
    links[["movieId", "imdb_id"]]
    .merge(tmdb_agg, on="imdb_id", how="left")
    .drop(columns=["imdb_id"])
)

print("movie_tmdb:", movie_tmdb.shape)
movie_tmdb.head()


movie_tmdb: (27278, 6)


,movieId,budget,revenue,adult,log1p_budget,log1p_revenue
0,1,30000000.0,394400000.0,0.0,17.216708,19.792876
1,2,65000000.0,262821940.0,0.0,17.989898,19.386987
2,3,25000000.0,71500000.0,0.0,17.034386,18.085208
3,4,16000000.0,81452156.0,0.0,16.588099,18.215526
4,5,NaN,76594107.0,0.0,NaN,18.154031


## 6) Merge features into train/val/test
These are **movie-level** features, so we left-join on `movieId`.

We also add missingness flags (often useful for tree models).

In [12]:
def add_tmdb_features(df: pd.DataFrame, movie_tmdb: pd.DataFrame) -> pd.DataFrame:
    out = df.merge(movie_tmdb, on="movieId", how="left")
    out["tmdb_has_budget"] = out["budget"].notna().astype("int8")
    out["tmdb_has_revenue"] = out["revenue"].notna().astype("int8")
    out["tmdb_has_adult"] = out["adult"].notna().astype("int8")
    return out

train3 = add_tmdb_features(train, movie_tmdb)
val3   = add_tmdb_features(val, movie_tmdb)
test3  = add_tmdb_features(test, movie_tmdb)

train3[["movieId","budget","revenue","adult","log1p_budget","log1p_revenue","tmdb_has_budget","tmdb_has_revenue","tmdb_has_adult"]].head()


,movieId,budget,revenue,adult,log1p_budget,log1p_revenue,tmdb_has_budget,tmdb_has_revenue,tmdb_has_adult
0,6163,NaN,5.126264e+06,0.0,NaN,15.449888,0,1,1
1,45722,200000000.0,1.065660e+09,0.0,19.113828,20.786860,1,1,1
2,53125,300000000.0,9.610000e+08,0.0,19.519293,20.683485,1,1,1
3,54259,70000000.0,1.375151e+08,0.0,18.064006,18.739245,1,1,1
4,316,55000000.0,1.965673e+08,0.0,17.822844,19.096515,1,1,1


## 7) Quick coverage check
We report what fraction of rows received TMDB values.

In [13]:
def coverage(df, col):
    return float(df[col].notna().mean())

pd.DataFrame({
    "split": ["train","val","test"],
    "budget_coverage": [coverage(train3,"budget"), coverage(val3,"budget"), coverage(test3,"budget")],
    "revenue_coverage": [coverage(train3,"revenue"), coverage(val3,"revenue"), coverage(test3,"revenue")],
    "adult_coverage": [coverage(train3,"adult"), coverage(val3,"adult"), coverage(test3,"adult")],
})


,split,budget_coverage,revenue_coverage,adult_coverage
0,train,0.896838,0.907310,0.996681
1,val,0.896414,0.907440,0.994375
2,test,0.897570,0.907984,0.993459


## 8) Persist updated datasets
Save enriched splits for downstream training.

In [14]:
train3.to_parquet("../data/processed/train_prepared_v3_tmdb.parquet", index=False)
val3.to_parquet("../data/processed/val_prepared_v3_tmdb.parquet", index=False)
test3.to_parquet("../data/processed/test_prepared_v3_tmdb.parquet", index=False)

print("Saved: train/val/test_prepared_v3_tmdb.parquet")


Saved: train/val/test_prepared_v3_tmdb.parquet


## 9) Quick Notes

The idea is to replace external sources via this dataset, which has a lot of useful information.
If needed, this information has to be accesed thorugh API or licence. So, thats something to consider.